In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import warnings

warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
df = pd.read_csv('student_dataset_updated.csv')


subject_cols = ['Maths', 'Science', 'English']
df['Avg_Score'] = df[subject_cols].mean(axis=1).round(1)

app = Dash(__name__)

app.layout = html.Div(
    style={
        'fontFamily': 'Inter, sans-serif',
        'backgroundColor': '#f7f7f7',
        'padding': '20px',
        'minHeight': '100vh'
    },
    children=[
        html.H1(
            children='Student Performance Interactive Dashboard',
            style={
                'textAlign': 'center',
                'color': '#1f2937',
                'marginBottom': '30px',
                'fontSize': '2.5rem'
            }
        ),
        html.Div(
            style={
                'display': 'flex',
                'flexDirection': 'column',
                'alignItems': 'center',
                'padding': '20px',
                'backgroundColor': 'white',
                'borderRadius': '12px',
                'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)'
            },
            children=[
                html.Label('Select Gender for Filtering:', style={'fontSize': '1.1rem', 'marginBottom': '10px', 'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='gender-filter-dropdown',
                    options=[
                        {'label': 'All Genders', 'value': 'All'},
                        {'label': 'Male (M)', 'value': 'M'},
                        {'label': 'Female (F)', 'value': 'F'}
                    ],
                    value='All',
                    clearable=False,
                    style={'width': '300px', 'borderRadius': '8px', 'borderColor': '#ccc'}
                )
            ]
        ),
        html.Div(
            style={'marginTop': '30px'},
            children=[
                html.Div(
                    style={'marginBottom': '30px'},
                    children=dcc.Graph(id='score-distribution-chart', style={'borderRadius': '12px', 'overflow': 'hidden'})
                ),
                html.Div(
                    style={
                        'display': 'grid',
                        'gridTemplateColumns': '1fr 1fr',
                        'gap': '20px',
                        '@media (max-width: 768px)': {
                            'gridTemplateColumns': '1fr',
                        }
                    },
                    children=[
                        html.Div(
                            children=dcc.Graph(id='study-vs-score-chart', style={'borderRadius': '12px', 'overflow': 'hidden'})
                        ),
                        html.Div(
                            children=dcc.Graph(id='performance-metrics-chart', style={'borderRadius': '12px', 'overflow': 'hidden'})
                        )
                    ]
                )
            ]
        )
    ]
)

@app.callback(
    Output('score-distribution-chart', 'figure'),
    Output('study-vs-score-chart', 'figure'),
    Output('performance-metrics-chart', 'figure'),
    Input('gender-filter-dropdown', 'value')
)
def update_graphs(selected_gender):
    if selected_gender == 'All':
        filtered_df = df
        filter_title = "All Students"
    else:
        filtered_df = df[df['Gender'] == selected_gender]
        filter_title = f"{'Male' if selected_gender == 'M' else 'Female'} Students"

    fig_hist = px.histogram(
        filtered_df,
        x='Avg_Score',
        color='Performance',
        nbins=10,
        title=f'Distribution of Average Scores for {filter_title}',
        marginal="box",
        height=500,
        color_discrete_map={'High': '#10b981', 'Medium': '#f59e0b'},
        hover_data=df.columns
    )
    fig_hist.update_layout(
        xaxis_title="Average Subject Score (0-100)",
        yaxis_title="Count of Students",
        plot_bgcolor='white',
        paper_bgcolor='#f7f7f7',
        margin={"t": 50, "b": 20, "l": 20, "r": 20}
    )

    fig_scatter = px.scatter(
        filtered_df,
        x='StudyHours',
        y='Attendance(%)',
        size='Avg_Score',
        color='Performance',
        title=f'Study Hours vs. Attendance (%) for {filter_title}',
        hover_name='Name',
        height=400,
        color_discrete_map={'High': '#10b981', 'Medium': '#f59e0b'}
    )
    fig_scatter.update_layout(
        xaxis_title="Study Hours Per Week",
        yaxis_title="Attendance (%)",
        plot_bgcolor='white',
        paper_bgcolor='#f7f7f7',
        margin={"t": 50, "b": 20, "l": 20, "r": 20}
    )

    performance_counts = filtered_df.groupby('Performance').size().reset_index(name='Count')

    fig_bar = px.bar(
        performance_counts,
        x='Performance',
        y='Count',
        color='Performance',
        title=f'Student Count by Performance Category for {filter_title}',
        height=400,
        color_discrete_map={'High': '#10b981', 'Medium': '#f59e0b'}
    )
    fig_bar.update_layout(
        xaxis_title="Performance Category",
        yaxis_title="Number of Students",
        xaxis={'categoryorder':'array', 'categoryarray':['High', 'Medium', 'Low']},
        plot_bgcolor='white',
        paper_bgcolor='#f7f7f7',
        margin={"t": 50, "b": 20, "l": 20, "r": 20}
    )

    return fig_hist, fig_scatter, fig_bar

if __name__ == "__main__":
    app.run(port=8050, debug=True)

<IPython.core.display.Javascript object>

In [ ]:
!pip install dash